In [1]:
import torch.nn as nn
from torchvision.models.segmentation import deeplabv3_resnet101

/home/misango/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
!pip install network

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/evillimiter-1.5.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Preparing metadata (setup.py) ... done
  Created wheel for network: filename=network-0.1-py3-none-any.whl size=3137 sha256=94237b2397ae6f4a475d5830e19fe8fe695464cff1823b7c2862ed42f6d07e92
  Stored in directory: /home/misango/.cache/pip/wheels/3a/9a/a4/341d3b109494a43a5cdd444ca83be3a4bfe8c1267ad9f85332
Successfully built network


In [4]:
!pip install utils

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/evillimiter-1.5.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13905 sha256=3598cca832c1d88ecfcb448022ecf50db548b47298092962b3f42e114fb9305d
  Stored in directory: /home/misango/.cache/pip/wheels/15/0c/b3/674aea8c5d91c642c817d4d630bd58faa316724b136844094d
Successfully built utils


In [ ]:
import os
import json
from PIL import Image
import numpy as np
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, root_dir, mode='train', transform=None):
        self.root_dir ='/home/misango/code/Architecture_ReadableML/Test_Run_CRF/image'
        self.transform = transform
        self.mode = mode
        self.data = self.load_data()

    def __len__(self):
        return len(self.data)
  
    def __getitem__(self, idx):
          img_path = os.path.join(self.root_dir, self.data['info']['image'])
          image = Image.open(img_path).convert('RGB')

          label_path = os.path.join(self.root_dir, self.data['info']['annotation'])
          label = Image.open(label_path)

          if self.transform:
              image = self.transform(image)
              label = self.transform(label)

          return image, label



    def load_data(self):
        # Load JSON file containing image paths and annotations
        with open(os.path.join(self.root_dir, '/home/misango/code/Architecture_ReadableML/Test_Run_CRF/image/November_2023 copy.json')) as f:
            data = json.load(f)
        return data

    def load_annotation(self, idx):
        label_path = os.path.join(self.root_dir, self.data[idx]['annotation'])
        label = Image.open(label_path)
        label = np.array(label)
        return torch.from_numpy(label)

    def get_num_classes(self):
      unique_labels = set()
      for item in self.data.values():
          if 'annotation' in item:  # Check if 'annotation' key exists in the current item
              label_path = os.path.join(self.root_dir, item['annotation'])
              label = Image.open(label_path)
              label = np.array(label)
              unique_labels.update(np.unique(label))
      return len(unique_labels)
    def segmentation_to_tensor(self, segmentation):
        # Convert segmentation to binary mask (2D numpy array)
        mask = np.zeros((height, width), dtype=np.uint8)
        for point in segmentation:
            mask[int(point[1]), int(point[0])] = 1

        # Convert binary mask to tensor
        return torch.from_numpy(mask)


In [ ]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
import torchvision.models as models


deep_learning_model = deeplabv3_resnet101
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define hyperparameters
batch_size = 32
learning_rate = 0.001
num_epochs = 10

# Define data transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    # Add more transformations as needed (e.g., normalization, data augmentation)
])

# Load your dataset
train_dataset = CustomDataset(transform=transform, mode='train', root_dir = '/home/misango/code/Architecture_ReadableML/Test_Run_CRF/image/November_2023.png')
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize your DeepV3 model
model = deep_learning_model(num_classes=train_dataset.get_num_classes()).to(device)


# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)

         # Ensure that outputs are tensors
        if isinstance(outputs, dict):
            outputs = outputs['out']

        # Calculate loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_dataset)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}')




In [51]:
from PIL import Image
from torchvision import transforms
input_image = Image.open('/content/November_2022.png')
input_image = input_image.convert("RGB")
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model


NameError: name 'output' is not defined

In [52]:
palette = torch.tensor([2 ** 25 - 1, 2 ** 15 - 1, 2 ** 21 - 1])
colors = torch.as_tensor([i for i in range(21)])[:, None] * palette
colors = (colors % 255).numpy().astype("uint8")

# plot the semantic segmentation predictions of 21 classes in each color
r = Image.fromarray(output_predictions.byte().cpu().numpy()).resize('/content/November_2022.png'.size)
r.putpalette(colors)

import matplotlib.pyplot as plt
plt.imshow(r)

NameError: name 'output_predictions' is not defined

In [ ]:
# Save the trained model
torch.save(model.state_dict(), 'deepv3_model.pth')